In [5]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [6]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.078666,-0.045802,-0.161414,0.110558,0.037687,0.150334,-0.002498,0.082596,-0.007764,-0.083418,...,-0.074862,-0.004713,-0.021268,0.256981,-0.130630,0.106356,0.061046,1,8077,critical
1,-0.020775,0.036146,-0.335274,0.212648,-0.035783,0.184416,0.209974,0.150083,0.036432,-0.120015,...,0.032618,0.237237,-0.105299,0.229636,0.026133,0.267945,-0.175635,1,5553,critical
2,-0.134288,-0.036787,-0.127596,0.135693,0.027834,0.077815,0.008317,0.092802,0.009843,-0.121727,...,-0.028849,0.006600,0.136421,-0.064934,0.206761,-0.001114,0.353806,1,9341,critical
3,-0.032318,-0.090595,-0.082224,0.146496,-0.017330,0.028888,-0.059263,0.038653,0.052420,-0.008678,...,-0.066082,0.212269,-0.066664,0.030977,0.072794,0.288604,-0.158939,1,10419,critical
4,-0.077614,-0.044658,-0.158415,0.108990,0.041627,0.138106,-0.004088,0.092520,-0.016285,-0.078490,...,-0.018110,0.252501,-0.136523,0.109989,0.062893,0.293013,-0.095985,1,12232,critical


In [7]:
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [8]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.7675
F1_macro: 0.5630
Accuracy: 0.7675
Precision_macro: 0.7919
Recall_macro: 0.5731

Bagging Metrics:
F1_micro: 0.7605
F1_macro: 0.5315
Accuracy: 0.7605
Precision_macro: 0.8046
Recall_macro: 0.5549

XGBoost Metrics:
F1_micro: 0.7799
F1_macro: 0.6200
Accuracy: 0.7799
Precision_macro: 0.7733
Recall_macro: 0.6100


In [9]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.7556
F1_macro: 0.5414
Precision_macro: 0.7361
Recall_macro: 0.5580
Accuracy: 0.7556
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.98      0.85       814
           1       0.71      0.14      0.23       295

    accuracy                           0.76      1109
   macro avg       0.74      0.56      0.54      1109
weighted avg       0.75      0.76      0.69      1109


Bagging Metrics:
F1_micro: 0.7574
F1_macro: 0.5247
Precision_macro: 0.7983
Recall_macro: 0.5506
Accuracy: 0.7574
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.99      0.86       814
           1       0.84      0.11      0.19       295

    accuracy                           0.76      1109
   macro avg       0.80      0.55      0.52      1109
weighted avg       0.78      0.76      0.68      1109


XGBoost Metrics:
F1_micro: 0.7737
F1_macro: 

In [11]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/base/result_codeT5_base_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/base/result_codeT5_base_avg.pkl
